In [1]:
from ETL.todoist.TodoIstExtractor import TodoIstExtractor
from Env.paths import *
import os
import pandas as pd


def is_directory_empty(directory_path):
    return len(os.listdir(directory_path)) == 0

def get_data(todoextractor: TodoIstExtractor) -> tuple:
    
    if is_directory_empty(EXTRACTED_DATA_PATH):
        data = todoextractor.extract_data()
        todoextractor.save_data_locally(data['opened_tasks'], 'opened_tasks.csv')
        todoextractor.save_data_locally(data['completed_tasks'], 'completed_tasks.csv')
        todoextractor.save_data_locally(data['projects'], 'projects.csv')
        todoextractor.save_data_locally(data['comments'], 'comments.csv')
        todoextractor.save_data_locally(data['productivity_stats'], 'productivity_stats.csv')

    files = [f for f in os.listdir(EXTRACTED_DATA_PATH) if os.path.isfile(os.path.join(EXTRACTED_DATA_PATH, f))]
    dfs = [ pd.read_pickle(os.path.join(EXTRACTED_DATA_PATH, f)) for f in files ]
    
    return files, dfs

In [2]:
extractor = TodoIstExtractor()
files, dfs = get_data(extractor)

In [3]:
dfs[0]

,color,comment_count,id,is_favorite,is_inbox_project,is_shared,is_team_inbox,name,order,parent_id,url,view_style,extraction_datetime
0,grey,0,2298360468,False,True,False,False,Inbox,0,None,https://todoist.com/showProject?id=2298360468,list,2023-12-25 06:48:10
1,orange,22,2299428438,True,False,False,False,📅 ToDo Week,1,None,https://todoist.com/showProject?id=2299428438,list,2023-12-25 06:48:10
2,blue,6,2316085252,True,False,False,False,📖 Versículos para ler,3,None,https://todoist.com/showProject?id=2316085252,list,2023-12-25 06:48:10
3,red,0,2321949037,True,False,False,False,🤖 My Routine Assistant,4,None,https://todoist.com/showProject?id=2321949037,board,2023-12-25 06:48:10
4,berry_red,1,2315713112,False,False,False,False,Energy Demmand Prediction ⚡🧑‍🔬📈,5,None,https://todoist.com/showProject?id=2315713112,board,2023-12-25 06:48:10
5,yellow,0,2315713212,False,False,False,False,🤓 📖Books Playlist,6,None,https://todoist.com/showProject?id=2315713212,list,2023-12-25 06:48:10


In [4]:

dfs[1].head(3)

,completed_at,content,id,item_object,meta_data,note_count,notes,project_id,section_id,task_id,user_id,extraction_datetime
0,2023-12-25 04:31:17-03:00,Definir as tarefas de amanhã,6916771756,None,None,0,[],2298360468,None,7054394305,41000232,2023-12-25 06:48:10
1,2023-12-25 04:31:13-03:00,Despertar + meditar na palavra + tomar café,6916771683,None,None,0,[],2298360468,None,6917637603,41000232,2023-12-25 06:48:10
2,2023-12-25 04:31:09-03:00,Tirar o lixo,6916771604,None,None,0,[],2298360468,None,6913897305,41000232,2023-12-25 06:48:10


In [5]:
dfs[2]

,assignee_id,assigner_id,comment_count,is_completed,content,created_at,creator_id,description,due,id,labels,order,parent_id,priority,project_id,section_id,url,sync_id,extraction_datetime
0,None,None,0,False,Whatch the video: How to use toDoist,2022-09-23 19:51:32.109522-03:00,41000232,[How I Use Todoist - Task & Project Management...,None,6202273884,[],1,None,1,2298360468,None,https://todoist.com/showTask?id=6202273884,None,2023-12-25 06:48:10
1,None,None,0,False,Tirar o lixo,2023-05-26 18:58:12.901855-03:00,41000232,Primeira coisa a fazer na hora do almoço,"{'date': '2023-12-26', 'is_recurring': True, '...",6913897305,[],11,None,1,2298360468,None,https://todoist.com/showTask?id=6913897305,None,2023-12-25 06:48:10
2,None,None,0,False,Despertar + meditar na palavra + tomar café,2023-05-28 20:04:59.491308-03:00,41000232,Duração de1H,"{'date': '2023-12-26', 'is_recurring': True, '...",6917637603,[],16,None,1,2298360468,None,https://todoist.com/showTask?id=6917637603,None,2023-12-25 06:48:10
3,None,None,0,False,Encontrar um bom banco de dados de demanda ene...,2023-07-05 18:44:09.144258-03:00,41000232,,None,7024838788,[],2,None,1,2315713112,127987968,https://todoist.com/showTask?id=7024838788,None,2023-12-25 06:48:10
4,None,None,0,False,Analisar cases envolvendo análise de series te...,2023-07-05 18:44:09.393276-03:00,41000232,,None,7024838794,[],3,None,1,2315713112,127987968,https://todoist.com/showTask?id=7024838794,None,2023-12-25 06:48:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,None,None,0,False,Seção Leitura,2023-12-23 06:09:28.445697-03:00,41000232,Total de 4 Pontos,None,7527788126,[🤓📖 Reading & Gettin extra Knowledge],21,7360214008,1,2299428438,122944882,https://todoist.com/showTask?id=7527788126,None,2023-12-25 06:48:10
94,None,None,0,False,Seção Leitura,2023-12-23 06:09:32.736934-03:00,41000232,Total de 4 Pontos,None,7527788239,[🤓📖 Reading & Gettin extra Knowledge],20,7360214008,1,2299428438,122944882,https://todoist.com/showTask?id=7527788239,None,2023-12-25 06:48:10
95,None,None,0,False,Seção Leitura,2023-12-23 06:09:37.173450-03:00,41000232,Total de 4 Pontos,None,7527788372,[🤓📖 Reading & Gettin extra Knowledge],19,7360214008,1,2299428438,122944882,https://todoist.com/showTask?id=7527788372,None,2023-12-25 06:48:10
96,None,None,0,False,Seção Leitura,2023-12-23 06:09:40.854164-03:00,41000232,Total de 4 Pontos,None,7527788480,[🤓📖 Reading & Gettin extra Knowledge],18,7360214008,1,2299428438,122944882,https://todoist.com/showTask?id=7527788480,None,2023-12-25 06:48:10


In [6]:
dfs[1][dfs[1]['project_id'] == '2298360468']

,completed_at,content,id,item_object,meta_data,note_count,notes,project_id,section_id,task_id,user_id,extraction_datetime
0,2023-12-25 04:31:17-03:00,Definir as tarefas de amanhã,6916771756,None,None,0,[],2298360468,None,7054394305,41000232,2023-12-25 06:48:10
1,2023-12-25 04:31:13-03:00,Despertar + meditar na palavra + tomar café,6916771683,None,None,0,[],2298360468,None,6917637603,41000232,2023-12-25 06:48:10
2,2023-12-25 04:31:09-03:00,Tirar o lixo,6916771604,None,None,0,[],2298360468,None,6913897305,41000232,2023-12-25 06:48:10
3,2023-12-25 04:31:05-03:00,Arrumar a cama,6916771533,None,None,0,[],2298360468,None,7045964169,41000232,2023-12-25 06:48:10
4,2023-12-25 04:31:02-03:00,Limpar a caixa de areia do gato,6916771490,None,None,0,[],2298360468,None,7055949593,41000232,2023-12-25 06:48:10
5,2023-12-23 05:36:36-03:00,Definir as tarefas de amanhã,6913166403,None,None,0,[],2298360468,None,7054394305,41000232,2023-12-25 06:48:10
6,2023-12-23 05:36:33-03:00,Despertar + meditar na palavra + tomar café,6913166345,None,None,0,[],2298360468,None,6917637603,41000232,2023-12-25 06:48:10
7,2023-12-22 18:46:46-03:00,Arrumar a cama,6912414600,None,None,0,[],2298360468,None,7045964169,41000232,2023-12-25 06:48:10
8,2023-12-21 20:35:10-03:00,Despertar + meditar na palavra + tomar café,6909701098,None,None,0,[],2298360468,None,6917637603,41000232,2023-12-25 06:48:10
9,2023-12-21 20:35:08-03:00,Tirar o lixo,6909701042,None,None,0,[],2298360468,None,6913897305,41000232,2023-12-25 06:48:10


In [7]:
dfs[2][dfs[2]['id'].isin(dfs[1]['task_id'].tolist())]

,assignee_id,assigner_id,comment_count,is_completed,content,created_at,creator_id,description,due,id,labels,order,parent_id,priority,project_id,section_id,url,sync_id,extraction_datetime
1,None,None,0,False,Tirar o lixo,2023-05-26 18:58:12.901855-03:00,41000232,Primeira coisa a fazer na hora do almoço,"{'date': '2023-12-26', 'is_recurring': True, '...",6913897305,[],11,None,1,2298360468,None,https://todoist.com/showTask?id=6913897305,None,2023-12-25 06:48:10
2,None,None,0,False,Despertar + meditar na palavra + tomar café,2023-05-28 20:04:59.491308-03:00,41000232,Duração de1H,"{'date': '2023-12-26', 'is_recurring': True, '...",6917637603,[],16,None,1,2298360468,None,https://todoist.com/showTask?id=6917637603,None,2023-12-25 06:48:10
19,None,None,0,False,Arrumar a cama,2023-07-13 07:29:01.496848-03:00,41000232,,"{'date': '2023-12-26', 'is_recurring': True, '...",7045964169,[],48,None,1,2298360468,None,https://todoist.com/showTask?id=7045964169,None,2023-12-25 06:48:10
21,None,None,0,False,Definir as tarefas de amanhã,2023-07-16 18:07:50.653799-03:00,41000232,,"{'date': '2023-12-26', 'is_recurring': True, '...",7054394305,[],51,None,1,2298360468,None,https://todoist.com/showTask?id=7054394305,None,2023-12-25 06:48:10
22,None,None,0,False,Limpar a caixa de areia do gato,2023-07-17 07:54:31.991068-03:00,41000232,,"{'date': '2023-12-31', 'is_recurring': True, '...",7055949593,[],55,None,1,2298360468,None,https://todoist.com/showTask?id=7055949593,None,2023-12-25 06:48:10
32,None,None,0,False,Planning da Semana,2023-08-05 05:10:20.379864-03:00,41000232,Total de 4 Pontos\nRealização da avaliação da ...,"{'date': '2023-12-30', 'is_recurring': True, '...",7110000353,[🤔 Planning],25,None,1,2299428438,122944882,https://todoist.com/showTask?id=7110000353,None,2023-12-25 06:48:10


In [8]:
dfs[2].head(3)

,assignee_id,assigner_id,comment_count,is_completed,content,created_at,creator_id,description,due,id,labels,order,parent_id,priority,project_id,section_id,url,sync_id,extraction_datetime
0,None,None,0,False,Whatch the video: How to use toDoist,2022-09-23 19:51:32.109522-03:00,41000232,[How I Use Todoist - Task & Project Management...,None,6202273884,[],1,None,1,2298360468,None,https://todoist.com/showTask?id=6202273884,None,2023-12-25 06:48:10
1,None,None,0,False,Tirar o lixo,2023-05-26 18:58:12.901855-03:00,41000232,Primeira coisa a fazer na hora do almoço,"{'date': '2023-12-26', 'is_recurring': True, '...",6913897305,[],11,None,1,2298360468,None,https://todoist.com/showTask?id=6913897305,None,2023-12-25 06:48:10
2,None,None,0,False,Despertar + meditar na palavra + tomar café,2023-05-28 20:04:59.491308-03:00,41000232,Duração de1H,"{'date': '2023-12-26', 'is_recurring': True, '...",6917637603,[],16,None,1,2298360468,None,https://todoist.com/showTask?id=6917637603,None,2023-12-25 06:48:10


In [9]:
dfs[3].head(3)

,date,items,total_completed,extraction_datetime
0,2023-12-24 21:00:00-03:00,"[{'completed': 5, 'id': '2298360468'}, {'compl...",8,2023-12-25 06:48:10
1,2023-12-23 21:00:00-03:00,"[{'completed': 4, 'id': '2299428438'}, {'compl...",5,2023-12-25 06:48:10
2,2023-12-22 21:00:00-03:00,"[{'completed': 2, 'id': '2298360468'}, {'compl...",5,2023-12-25 06:48:10


In [10]:
for i in range(len(dfs[4]) -1):
    
    if dfs[4]['attachment'][i+1]:
        check = list(dfs[4]['attachment'][i].keys()) == list(dfs[4]['attachment'][i + 1].keys())
        if check == False:
            print('error')
            break
print('no error, eschema are fixed')

no error, eschema are fixed


In [11]:
[k for k in list(dfs[1]['notes'])  if len(k) !=0]

[]

In [12]:
[ i for i in list(dfs[1]['meta_data']) if i]

[]

In [13]:
[i for i in dfs[2]['labels'].to_list() if i]

[['Não importante e Urgente'],
 ['Não importante e Urgente'],
 ['Database'],
 ['Database'],
 ['Database'],
 ['Database'],
 ['Database'],
 ['Database'],
 ['Database'],
 ['Database'],
 ['🤔 Planning'],
 ['🤓📖 Reading & Gettin extra Knowledge'],
 ['Health ⚕️🏋🏼'],
 ['🔎 Staying up-to-date'],
 ['🚀 Personal Project  🗺️ 📐 🛠️'],
 ['🔎 Staying up-to-date'],
 ['General'],
 ['General'],
 ['General'],
 ['General'],
 ['Health ⚕️🏋🏼'],
 ['Data Engineering'],
 ['Data Engineering'],
 ['Data Engineering'],
 ['Data Engineering'],
 ['Data Engineering'],
 ['Data Engineering'],
 ['Data Analytics'],
 ['Data Analytics'],
 ['Database'],
 ['Database'],
 ['GenAI'],
 ['GenAI'],
 ['GenAI'],
 ['GenAI'],
 ['🤓📖 Reading & Gettin extra Knowledge'],
 ['🤓📖 Reading & Gettin extra Knowledge'],
 ['🤓📖 Reading & Gettin extra Knowledge'],
 ['🤓📖 Reading & Gettin extra Knowledge'],
 ['🤓📖 Reading & Gettin extra Knowledge'],
 ['🤓📖 Reading & Gettin extra Knowledge']]

In [14]:
sum(dfs[2]['assigner_id']  != None)

98

In [15]:
list(dfs[4]['attachment'])

[{'resource_type': 'image',
  'file_name': 'image.png',
  'file_size': 76194,
  'file_type': 'image/png',
  'file_url': 'https://files.todoist.com/hhTqMv04xzQzmrxc6-lfVVoY5u43SBj25jeC9eCdUzUW_JAiJAJ06fHKlvE1apXW/by/41000232/as/file.png',
  'file_duration': None,
  'upload_state': 'completed',
  'image': 'https://files.todoist.com/hhTqMv04xzQzmrxc6-lfVVoY5u43SBj25jeC9eCdUzUW_JAiJAJ06fHKlvE1apXW/by/41000232/as/file.png',
  'image_width': 1259,
  'image_height': 530,
  'url': None,
  'title': None},
 {'resource_type': 'image',
  'file_name': 'image.png',
  'file_size': 3329,
  'file_type': 'image/png',
  'file_url': 'https://files.todoist.com/9wp6WWCh0GutaqWJocR8MsXeIUSH_n185TOya2iNnKilXrn2fldo1QRi4wgMEuQC/by/41000232/as/file.png',
  'file_duration': None,
  'upload_state': 'completed',
  'image': 'https://files.todoist.com/9wp6WWCh0GutaqWJocR8MsXeIUSH_n185TOya2iNnKilXrn2fldo1QRi4wgMEuQC/by/41000232/as/file.png',
  'image_width': 460,
  'image_height': 72,
  'url': None,
  'title': None},

In [16]:
cols1 = set(dfs[1].columns)
cols2 = set(dfs[2].columns)
cols2.difference(cols1)

{'assignee_id',
 'assigner_id',
 'comment_count',
 'created_at',
 'creator_id',
 'description',
 'due',
 'is_completed',
 'labels',
 'order',
 'parent_id',
 'priority',
 'sync_id',
 'url'}

In [17]:
len(dfs[1]['section_id'][80])

9

In [18]:
[i for i in list(dfs[1]['notes']) if i]

[]

In [19]:
len(dfs[1]['user_id'][0])

8

In [20]:
hex(hash('luacs'))

'0x5be5c4d4df27e746'

In [21]:
import uuid
 
# initializing a string
url = "https://files.todoist.com/I5P1nKsrwL1UAd9WQJBFC4jQNJJyy7RFm0FxijxQMZcv0WFIyQUxJ54wqXwy55d8/by/41000232/as/file.png"
 
# using NAMESPACE_URL as namespace 
# to generate MD5 hash uuid
print ("The SHA1 hash value generated ID is : ", 
            uuid.uuid3(uuid.NAMESPACE_URL, url))
 
# using NAMESPACE_URL as namespace  
# to generate SHA-1 hash uuid
print ("The MD5 hash value generated ID is : ", 
            uuid.uuid5(uuid.NAMESPACE_URL, url))

The SHA1 hash value generated ID is :  923ff989-05f5-3448-a022-7457f4bc4bdd
The MD5 hash value generated ID is :  f94b53ca-a8af-54d7-8bc6-d3e72d1803ad


In [22]:
from pymongo import MongoClient
import datetime

client = MongoClient("mongodb://localhost:27017/")
db = client["test-database"]
collection = db["test-collection"]
post = {

    "author": "Mike",

    "text": "My first blog post!",

    "tags": ["mongodb", "python", "pymongo"],

    "date": datetime.datetime.now(tz=datetime.timezone.utc),

}
post_id = collection.insert_one(post).inserted_id
post_id

ObjectId('658ebeb7c4d130d1b983b4bf')

In [23]:
import pprint

pprint.pprint(collection.find_one())

{'_id': ObjectId('658eb6b3eaad2b415e72ec6f'),
 'author': 'Mike',
 'date': datetime.datetime(2023, 12, 29, 12, 8, 19, 447000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [24]:
db.list_collection_names()

['test-collection2', 'test-collection']

In [25]:
from DatabaseHandler.MongoDBHandler import MongoDBHandler

mongodb = MongoDBHandler()
mongodb.connect('test-database')
mongodb.read_data('test-collection', {'text': 'My first blog post!'})

[{'_id': ObjectId('658eb6b3eaad2b415e72ec6f'),
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetime(2023, 12, 29, 12, 8, 19, 447000)},
 {'_id': ObjectId('658eb6b8eaad2b415e72ec71'),
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetime(2023, 12, 29, 12, 8, 24, 103000)},
 {'_id': ObjectId('658ebca798f9f1cff8f23f83'),
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetime(2023, 12, 29, 12, 33, 43, 223000)},
 {'_id': ObjectId('658ebd6922369a9551d5abaa'),
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetime(2023, 12, 29, 12, 36, 57, 162000)},
 {'_id': ObjectId('658ebeb7c4d130d1b983b4bf'),
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetim

In [26]:
mongodb.write_data('test-collection2', [{'data': 'this is some data for test reasons'}])
mongodb.read_data('test-collection2', {})

[{'_id': ObjectId('658ebe4d22369a9551d5abac'),
  'data': 'this is some data for test reasons'},
 {'_id': ObjectId('658ebeb8c4d130d1b983b4c1'),
  'data': 'this is some data for test reasons'}]

In [27]:
mongodb.disconnect()

In [28]:
dfs[4].to_dict('records')

[{'attachment': {'resource_type': 'image',
   'file_name': 'image.png',
   'file_size': 76194,
   'file_type': 'image/png',
   'file_url': 'https://files.todoist.com/hhTqMv04xzQzmrxc6-lfVVoY5u43SBj25jeC9eCdUzUW_JAiJAJ06fHKlvE1apXW/by/41000232/as/file.png',
   'file_duration': None,
   'upload_state': 'completed',
   'image': 'https://files.todoist.com/hhTqMv04xzQzmrxc6-lfVVoY5u43SBj25jeC9eCdUzUW_JAiJAJ06fHKlvE1apXW/by/41000232/as/file.png',
   'image_width': 1259,
   'image_height': 530,
   'url': None,
   'title': None},
  'content': 'Ta esquisito, verificar o porque',
  'id': '3321888008',
  'posted_at': Timestamp('2023-07-18 10:43:26.618672-0300', tz='America/Sao_Paulo'),
  'project_id': None,
  'task_id': '7024838790',
  'extraction_datetime': Timestamp('2023-12-25 06:48:10')},
 {'attachment': {'resource_type': 'image',
   'file_name': 'image.png',
   'file_size': 3329,
   'file_type': 'image/png',
   'file_url': 'https://files.todoist.com/9wp6WWCh0GutaqWJocR8MsXeIUSH_n185TOya2iNnK